In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 25.7 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [ ]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [ ]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [ ]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2025-04-26 19:15:33,623] A new study created in memory with name: no-name-054b1fea-7507-4698-8b22-a1d95dcdfcd6
[I 2025-04-26 19:15:33,755] Trial 0 finished with value: 0.7858472998137801 and parameters: {'classifier': 'SVM', 'C': 30.974916687534307, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 0 with value: 0.7858472998137801.
[I 2025-04-26 19:15:35,039] Trial 1 finished with value: 0.7392923649906891 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 89, 'learning_rate': 0.018291133488171015, 'max_depth': 13, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.7858472998137801.
[I 2025-04-26 19:15:37,719] Trial 2 finished with value: 0.7541899441340782 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 245, 'learning_rate': 0.04452857821122006, 'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.7858472998137801.
[I 2025-04-26 19:15:37,762] Trial 3 finished with value: 0.71

In [ ]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.14779766088034818, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.756052,2025-04-26 19:14:00.839294,2025-04-26 19:14:01.955928,0 days 00:00:01.116634,NaN,True,RandomForest,NaN,NaN,NaN,15.0,10.0,5.0,200.0,COMPLETE
1,1,0.700186,2025-04-26 19:14:01.959332,2025-04-26 19:14:01.996020,0 days 00:00:00.036688,35.709452,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.769088,2025-04-26 19:14:01.998468,2025-04-26 19:14:02.992001,0 days 00:00:00.993533,NaN,False,RandomForest,NaN,NaN,NaN,13.0,6.0,10.0,159.0,COMPLETE
3,3,0.757914,2025-04-26 19:14:02.993541,2025-04-26 19:14:04.689523,0 days 00:00:01.695982,NaN,True,RandomForest,NaN,NaN,NaN,19.0,10.0,3.0,278.0,COMPLETE
4,4,0.748603,2025-04-26 19:14:04.690666,2025-04-26 19:14:08.370367,0 days 00:00:03.679701,NaN,NaN,GradientBoosting,NaN,NaN,0.123343,20.0,2.0,8.0,269.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.789572,2025-04-26 19:14:33.486164,2025-04-26 19:14:33.529492,0 days 00:00:00.043328,0.119095,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.761639,2025-04-26 19:14:33.530466,2025-04-26 19:14:35.457191,0 days 00:00:01.926725,NaN,NaN,GradientBoosting,NaN,NaN,0.020365,20.0,8.0,7.0,167.0,COMPLETE
97,97,0.785847,2025-04-26 19:14:35.459386,2025-04-26 19:14:35.529105,0 days 00:00:00.069719,0.215114,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.772812,2025-04-26 19:14:35.530379,2025-04-26 19:14:37.780860,0 days 00:00:02.250481,NaN,False,RandomForest,NaN,NaN,NaN,10.0,9.0,3.0,299.0,COMPLETE


In [ ]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,171
GradientBoosting,15
RandomForest,14


In [ ]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.755307
RandomForest,0.766028
SVM,0.779411
